# Pathological Parameter Regimes

This notebook explores how different HGF parameter settings can model computational abnormalities associated with psychiatric disorders.

**IMPORTANT**: These are theoretical models for research purposes. They do NOT constitute clinical diagnostic criteria.

## The Computational Psychiatry Hypothesis

Many psychiatric disorders may reflect imbalances in Bayesian inference:
- **Schizophrenia**: Aberrant precision weighting
- **BPD**: Hypersensitive volatility coupling
- **Anxiety**: Elevated threat precision
- **Depression**: Reduced precision/motivation
- **Autism**: Elevated sensory precision

In [ ]:
# Setup
import sys
sys.path.insert(0, '../..')

import numpy as np
import matplotlib.pyplot as plt

from ara.hgf import (
    HGFAgent,
    VolatilitySwitchingTask,
    plot_pathology_comparison,
    plot_hgf_dashboard,
)
from ara.hgf.pathology import (
    HEALTHY_BASELINE,
    SCHIZOPHRENIA_RIGID,
    SCHIZOPHRENIA_LOOSE,
    BPD_HIGH_KAPPA,
    ANXIETY_HIGH_PRECISION,
    list_presets,
    compare_presets,
)

plt.style.use('dark_background')
%matplotlib inline

## 1. Available Pathology Presets

In [ ]:
# List all presets
for code, name in list_presets():
    print(f"  {code}: {name}")

In [ ]:
# Compare key parameters
comparison = compare_presets(['HEALTHY', 'SCZ_RIGID', 'SCZ_LOOSE', 'BPD', 'ANXIETY'])

print("Parameter Comparison:")
print(f"{'Preset':<12} {'ω₂':<8} {'κ₁':<8} {'θ':<8}")
print("-" * 40)
for code in comparison['presets']:
    print(f"{code:<12} {comparison['omega_2'][code]:<8.1f} {comparison['kappa_1'][code]:<8.1f} {comparison['theta'][code]:<8.1f}")

## 2. The BPD Model: High κ₁

In Borderline Personality Disorder, the coupling parameter κ₁ may be pathologically elevated.

This means:
- Small changes in perceived volatility (μ₃) cause LARGE changes in prior variance (σ̂₂)
- Beliefs become hypersensitive to environmental fluctuations
- Emotional states swing rapidly based on minor events

$$\hat{\sigma}_2 = \sigma_2 + \exp(\kappa_1 \cdot \mu_3 + \omega_2)$$

When κ₁ = 3.0 (vs healthy 1.0), the exponential term amplifies volatility effects dramatically.

In [ ]:
# Create task
task = VolatilitySwitchingTask(n_trials=200, seed=42)
task_data = task.generate(seed=42)

# Create agents
healthy_agent = HEALTHY_BASELINE.create_agent()
bpd_agent = BPD_HIGH_KAPPA.create_agent()

# Run both
healthy_traj = healthy_agent.run(task_data)
bpd_traj = bpd_agent.run(task_data)

print(f"Healthy: κ₁ = {HEALTHY_BASELINE.params.kappa_1}")
print(f"BPD:     κ₁ = {BPD_HIGH_KAPPA.params.kappa_1}")

In [ ]:
# Compare belief dynamics
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

trials = np.arange(task_data.n_trials)

# Predictions
ax = axes[0]
ax.plot(trials, healthy_traj.get_predictions(), 'cyan', lw=2, label='Healthy')
ax.plot(trials, bpd_traj.get_predictions(), 'magenta', lw=2, label='BPD (high κ₁)')
ax.plot(trials, task_data.true_probabilities, 'orange', lw=1, linestyle='--', label='True', alpha=0.7)
ax.set_ylabel('Prediction')
ax.set_title('Predictions: BPD shows more volatile tracking')
ax.legend()

# Level 2 beliefs
ax = axes[1]
ax.plot(trials, healthy_traj.get_beliefs(2), 'cyan', lw=2, label='Healthy μ₂')
ax.plot(trials, bpd_traj.get_beliefs(2), 'magenta', lw=2, label='BPD μ₂')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('μ₂ (logit)')
ax.set_title('Hidden State: BPD shows larger swings')
ax.legend()

# Volatility (Level 3)
ax = axes[2]
ax.plot(trials, healthy_traj.get_beliefs(3), 'cyan', lw=2, label='Healthy μ₃')
ax.plot(trials, bpd_traj.get_beliefs(3), 'magenta', lw=2, label='BPD μ₃')
ax.set_xlabel('Trial')
ax.set_ylabel('μ₃ (log-volatility)')
ax.set_title('Volatility Estimate: BPD more reactive')
ax.legend()

plt.tight_layout()

## 3. Schizophrenia: Two Modes

Schizophrenia may manifest in two computational phenotypes:

### Rigid Priors (Negative Symptoms)
- Very low ω₂ = beliefs don't update
- Low κ₁ = ignores volatility signals
- Associated with cognitive inflexibility, flat affect

### Loose Priors (Positive Symptoms)
- High ω₂ = beliefs are unstable
- High κ₁ = over-sensitive to volatility
- Associated with hallucinations, delusions (finding patterns in noise)

In [ ]:
# Create agents
scz_rigid_agent = SCHIZOPHRENIA_RIGID.create_agent()
scz_loose_agent = SCHIZOPHRENIA_LOOSE.create_agent()

# Run
scz_rigid_traj = scz_rigid_agent.run(task_data)
scz_loose_traj = scz_loose_agent.run(task_data)

print("Schizophrenia (Rigid):")
print(f"  {SCHIZOPHRENIA_RIGID.mechanism}")
print()
print("Schizophrenia (Loose):")
print(f"  {SCHIZOPHRENIA_LOOSE.mechanism}")

In [ ]:
# Compare all three
trajectories = {
    'Healthy': healthy_traj,
    'SCZ (Rigid)': scz_rigid_traj,
    'SCZ (Loose)': scz_loose_traj,
}

fig = plot_pathology_comparison(trajectories, metric='beliefs', level=2)
plt.title('Level 2 Beliefs: Schizophrenia Comparison')
plt.show()

In [ ]:
# Compare prediction errors
fig, ax = plt.subplots(figsize=(14, 4))

colors = {'Healthy': 'cyan', 'SCZ (Rigid)': 'yellow', 'SCZ (Loose)': 'red'}

for label, traj in trajectories.items():
    delta_2 = traj.get_prediction_errors(2)
    ax.plot(trials, delta_2, color=colors[label], lw=1.5, label=label, alpha=0.8)

ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Trial')
ax.set_ylabel('δ₂ (volatility PE)')
ax.set_title('Volatility Prediction Errors')
ax.legend()
plt.tight_layout()

## 4. Clinical Implications

### The NeuroBalance Hypothesis

If these computational abnormalities are real, we could potentially:

1. **Diagnose** by fitting HGF parameters to behavioral tasks
2. **Treat** by modulating the neural circuits that implement these computations

For example, if BPD involves high κ₁:
- Measure frontal theta (Π_prior proxy) and posterior alpha (Π_sensory proxy)
- Apply targeted stimulation to dampen the κ₁ coupling
- Measure behavioral improvement via parameter shift toward healthy range

In [ ]:
# Print clinical correlates
print("BPD Clinical Correlates:")
for key, value in BPD_HIGH_KAPPA.clinical_correlates.items():
    print(f"  {key}: {value}")

print()
print("Mechanism:")
print(f"  {BPD_HIGH_KAPPA.mechanism}")

## 5. The Validation Ladder

To prove these hypotheses, we need:

1. **Behavioral Fit**: Does the model explain task performance?
2. **Parameter-Symptom Correlation**: Do fitted κ₁ values correlate with BPD severity?
3. **Neural Correlates**: Do δ₁/δ₂ terms match EEG/fMRI signals?
4. **Causal Modulation**: Can we shift κ₁ with stimulation?
5. **Clinical Outcome**: Does shifting κ₁ reduce symptoms?

This notebook (and the HGF package) provides the tools for steps 1-3.
Steps 4-5 require real subjects, ethics approval, and clinical oversight.

In [ ]:
# Summary comparison
print("\nSummary: Accuracy by Phenotype")
print("=" * 40)
for label, traj in trajectories.items():
    acc = traj.compute_accuracy()
    ll = traj.compute_log_likelihood()
    print(f"{label:<15} Accuracy: {acc:.3f}")